# Implementation of RNNs from scratch

Now we will implement a language model based on a character-level RNN trained on H. G. Wells’ *The Time Machine*. We start by reading the data set first

In [ ]:
import utils
import torch
import math
import d2l

In [ ]:
batch_size, num_steps = 32, 35
train_iter, vocab = # type your code here

In [ ]:
sample = # type your code here

In [ ]:
# type your code here

## One-hot Encoding

Remember that each token is presented as a numerical index in ``train_iter``. Feeding these indices directly to the neural network might make it hard to learn. We often present each token as a more expressive feature vector. The easiest presentation is called ``one-hot encoding``.

In a nutshell, we map each index to a different unit vector: assume that the number of different tokens in the vocabulary is N (the ``len(vocab)``) and the token indices range from 0 to N-1. If the index of a token is the integer i , then we create a vector $\mathbf{e}_i$ of all 0s with a length of N and set the element at position i to 1. This vector is the one-hot vector of the original token. The one-hot vectors with indices 0 and 2 are shown below.

In [ ]:
x = # type your code here
x_one_hot = # type your code here

In [ ]:
# type your code here

The shape of the mini-batch we sample each time is (batch size, time step). The ``one_hot`` function transforms such a mini-batch into a 3-D tensor with the last dimension equals to the vocabulary size. We often transpose the input so that we will obtain a (time step, batch size, vocabulary size) output that fits into a sequence
model easier.

In [ ]:
# type your code here

## Initializing the Model Parameters

Next, we initialize the model parameters for a RNN model. The number of hidden units ``num_hiddens`` is a tunable parameter.

In [ ]:
def get_params(vocab_size, num_hiddens):
    # type your code here
    # Hidden layer parameters
    # type your code here
    # Output layer parameters
    # type your code here
    # Attach a gradient
    params = # type your code here
    for param in params: 
        # type your code here
    return # type your code here

In [ ]:
# type your code here

In [ ]:
# type your code here

## RNN Model

First, we need an ``init_rnn_state`` function to return the hidden state at initialization. It returns a tuple consisting of a tensor with a value of 0 and a shape of (batch size, number of hidden units). Using tuples makes it easier to handle situations where the hidden state contains multiple variables (e.g. when combining multiple layers in an RNN where each layers requires initializing).

In [ ]:
def init_rnn_state(batch_size, num_hiddens):
    return # type your code here

The following ``rnn`` function defines how to compute the hidden state and output in a time step. The activation function here uses the ``tanh`` function.

In [ ]:
def rnn(inputs, state, params):
    # inputs shape: (num_steps, batch_size, vocab_size)
    # type your code here
    # type your code here
    outputs = # type your code here
    for X in inputs:
        H = # type your code here
        Y = # type your code here
        # type your code here
    return # type your code here

Now we have all functions defined, next we create a class to wrap these functions and store parameters.

In [ ]:
class RNNModelScratch(object):
    """A RNN Model based on scratch implementations"""
    def __init__(self, vocab_size, num_hiddens, get_params, init_state, forward):
        # type your code here
    def __call__(self, X, state):
        # type your code here
        return # type your code here
    def begin_state(self, batch_size):
        return # type your code here

Let’s do a sanity check whether inputs and outputs have the correct dimensions, e.g. to ensure that the dimensionality of the hidden state hasn’t changed.

In [ ]:
# type your code here

We can see that the output shape is (number steps x batch size, vocabulary size), while the state shape remains the same, i.e. (batch size, number of hidden units).

## Prediction

We first explain the predicting function so we can regularly check the prediction during training. This function predicts the next ``num_predicts`` characters based on the prefix (a string containing several characters). For the beginning of the sequence, we only update the hidden state. After that we begin generating new characters and emitting them.

In [ ]:
def predict_rnn(prefix, num_predicts, model, vocab):
    # type your code here
    for y in prefix[1:]: # Warmup state with prefix
        # type your code here
    for _ in range(num_predicts): # Predict num_predicts steps
        # type your code here
    return # type your code here

We test the ``predict_rnn`` function first. Given that we didn’t train the network it will generate nonsensical predictions. We initialize it with the sequence traveller and have it generate 10 additional characters.

In [ ]:
# type your code here

## Gradient Clipping

For a sequence of length T, we compute the gradients over these T time steps in an iteration, which results in a chain of matrix-products with length $O(T)$ during backpropagating. As we mentioned before, it might result in numerical instability, e.g. the gradients may either explode or vanish, when T is large. Therefore RNN models often need extra help to stabilize the training.

Sometimes the gradients can be quite large and the optimization algorithm may fail to converge. We could address this by reducing the learning rate $\eta$ or by some other higher order trick. But what if we only rarely get large gradients? In this case such an approach may appear entirely unwarranted. One alternative is to
clip the gradients by projecting them back to a ball of a given radius, say $\theta$ via $$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g} \|} \right) \mathbf{g}$$.

By doing so we know that the gradient norm never exceeds $\theta$ and that the updated gradient is entirely aligned with the original direction $\mathbf{g}$. This bestows a certain degree of robustness to the model. Gradient clipping provides a quick fix to the gradient exploding. While it doesn’t entirely solve the problem, it is one of the many techniques to alleviate it.

Below we define a function to clip the gradients of a model. Note that we compute the gradient norm over all parameters.

In [ ]:
def grad_clipping(model, theta):
    # type your code here

## Training

Let’s first define the function to train the model on one data epoch. It differs to the
models training from previous chapters in three places:

* Different sampling methods for sequential data (random sampling and sequential partitioning) will result in differences in the initialization of hidden states.

* We clip the gradient before updating the model parameters. This ensures that the model doesn’t diverge even when gradients blow up at some point during the training process (effectively it reduces the stepsize automatically).

* We use perplexity to evaluate the model. This ensures that different tests are comparable.

When the consecutive sampling is used, we initialize the hidden state at the beginning of each epoch. Since the i-th example in the next mini-batch is adjacent to the current i-th example, so the next mini-batch can use the current hidden state directly, we only detach the gradient so that we only compute the gradients within a mini-batch. When using the random sampling, we need to re-initialize the hidden state for each iteration since each example is sampled with a random position.

In [ ]:
def train_epoch_rnn(model, train_iter, loss, updater, use_random_iter):
    # type your code here
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # Initialize state when either it's the first iteration or
            # using random sampling.
            state = model.begin_state(batch_size=X.shape[0])
    else:
        for s in state: 
            # type your code here
    # type your code here
    return # type your code here

In [ ]:
def train_rnn(model, train_iter, vocab, lr, num_epochs, use_random_iter=False):
    # type your code here
    # Train and check the progress.
    for epoch in range(num_epochs):
        # type your code here
    # type your code here

Finally we can train a model. Since we only use 10,000 tokens in the dataset, so here we need more data epochs to converge.

In [ ]:
# type your code here